In [56]:
import pandas as pd
import glob
import os

def load_cointegration_csvs(folder_path):
    """
    Load all CSV files with filenames like 'SYM1_SYM2_bin_futures_window_cointegration.csv'
    into a dictionary {(sym1, sym2): DataFrame}
    """
    csv_files = glob.glob(os.path.join(folder_path, "*_bin_futures_window_cointegration.csv"))
    data_dict = {}

    for f in csv_files:
        base = os.path.basename(f).replace(".csv", "")
        try:
            # Extract symbols
            parts = base.split("_")
            sym1, sym2 = parts[0], parts[1]
        except Exception as e:
            print(f"Skipping file {f}, cannot parse symbols: {e}")
            continue

        # Read CSV
        df = pd.read_csv(f)
        data_dict[(sym1, sym2)] = df

    return data_dict

# Usage
folder_path = ""  # replace with your path
cointegration_dict = load_cointegration_csvs(folder_path)

# Check loaded keys
print(list(cointegration_dict.keys())[:10])


[('AAVE', 'SUI'), ('AAVE', 'TRX'), ('ADA', 'BTC'), ('ADA', 'DOGE'), ('ADA', 'HBAR'), ('ADA', 'LTC'), ('ADA', 'SUI'), ('ADA', 'XLM'), ('ADA', 'XRP'), ('APT', 'AVAX')]


In [57]:
# selection of top-5 pairs per window based on correlation
top_pairs_per_window = {}
for pair, df in cointegration_dict.items():
    for _, row in df.iterrows():
        window_key = (pd.to_datetime(row["start"] + " 00:00:00", format="%Y-%m-%d %H:%M:%S"), pd.to_datetime(row["end"], format="%Y-%m-%d %H:%M:%S"))
        if window_key not in top_pairs_per_window:
            top_pairs_per_window[window_key] = []
        if row["cointegrated"]:
            top_pairs_per_window[window_key].append((pair, row["correlation"]))
# keep top 5 pairs by correlation for each window
for window_key, pairs in top_pairs_per_window.items():
    # sort by absolute correlation
    pairs.sort(key=lambda x: abs(x[1]), reverse=True)  
    top_pairs_per_window[window_key] = pairs[:5]


In [58]:
top_pairs_per_window

{(Timestamp('2024-05-01 00:00:00'),
  Timestamp('2024-05-03 23:59:00')): [(('AVAX', 'ETC'),
   0.974180033509878), (('XLM', 'XRP'), 0.9594458707339524), (('ETC', 'ETH'),
   0.9444983486845284), (('ADA', 'XRP'), 0.942780162702516), (('ADA', 'LTC'),
   0.9414120954592604)],
 (Timestamp('2024-05-02 00:00:00'),
  Timestamp('2024-05-04 23:59:00')): [(('ETC', 'ETH'),
   0.9898628589983928), (('NEAR', 'WLD'), 0.984789694748536), (('ETC', 'NEAR'),
   0.9843443241193592), (('ARB', 'ETC'), 0.9821965704585714), (('ARB', 'ETH'),
   0.9807634222003347)],
 (Timestamp('2024-05-03 00:00:00'),
  Timestamp('2024-05-05 23:59:00')): [(('NEAR', 'WLD'),
   0.9588261862490038), (('DOGE', 'LINK'), 0.949398546006078), (('BNB',
    'LINK'),
   0.9466993551306644), (('ETC', 'NEAR'), 0.9396333100678572), (('ADA', 'LTC'),
   0.9209341085604732)],
 (Timestamp('2024-05-04 00:00:00'),
  Timestamp('2024-05-06 23:59:00')): [(('DOT', 'ETC'),
   0.9208620431182596), (('ARB', 'DOT'), 0.8786325245064255), (('ADA', 'LTC'),


In [59]:
rows = []
for (start, end), pairs in top_pairs_per_window.items():
    for (sym1, sym2), corr in pairs:
        rows.append({
            "start": start,
            "end": end,
            "sym1": sym1,
            "sym2": sym2,
            "correlation": corr
        })

df = pd.DataFrame(rows)
df.to_csv("top_pairs_per_window.csv", index=False)


In [60]:
# make sure that there are 5 pairs for each window
for window_key, pairs in top_pairs_per_window.items():
    if len(pairs) < 5:
        print(f"Warning: Only {len(pairs)} pairs found for window {window_key}")
# not always 5 pairs

In [61]:
top_pairs_per_window

{(Timestamp('2024-05-01 00:00:00'),
  Timestamp('2024-05-03 23:59:00')): [(('AVAX', 'ETC'),
   0.974180033509878), (('XLM', 'XRP'), 0.9594458707339524), (('ETC', 'ETH'),
   0.9444983486845284), (('ADA', 'XRP'), 0.942780162702516), (('ADA', 'LTC'),
   0.9414120954592604)],
 (Timestamp('2024-05-02 00:00:00'),
  Timestamp('2024-05-04 23:59:00')): [(('ETC', 'ETH'),
   0.9898628589983928), (('NEAR', 'WLD'), 0.984789694748536), (('ETC', 'NEAR'),
   0.9843443241193592), (('ARB', 'ETC'), 0.9821965704585714), (('ARB', 'ETH'),
   0.9807634222003347)],
 (Timestamp('2024-05-03 00:00:00'),
  Timestamp('2024-05-05 23:59:00')): [(('NEAR', 'WLD'),
   0.9588261862490038), (('DOGE', 'LINK'), 0.949398546006078), (('BNB',
    'LINK'),
   0.9466993551306644), (('ETC', 'NEAR'), 0.9396333100678572), (('ADA', 'LTC'),
   0.9209341085604732)],
 (Timestamp('2024-05-04 00:00:00'),
  Timestamp('2024-05-06 23:59:00')): [(('DOT', 'ETC'),
   0.9208620431182596), (('ARB', 'DOT'), 0.8786325245064255), (('ADA', 'LTC'),


In [62]:
# make top_pairs_per_window only contain pairs, not correlation
top_pairs_per_window = {k: [p[0] for p in v] for k, v in top_pairs_per_window.items()}

In [63]:
top_pairs_per_window

{(Timestamp('2024-05-01 00:00:00'),
  Timestamp('2024-05-03 23:59:00')): [('AVAX', 'ETC'), ('XLM', 'XRP'), ('ETC',
   'ETH'), ('ADA', 'XRP'), ('ADA', 'LTC')],
 (Timestamp('2024-05-02 00:00:00'),
  Timestamp('2024-05-04 23:59:00')): [('ETC', 'ETH'), ('NEAR', 'WLD'), ('ETC',
   'NEAR'), ('ARB', 'ETC'), ('ARB', 'ETH')],
 (Timestamp('2024-05-03 00:00:00'),
  Timestamp('2024-05-05 23:59:00')): [('NEAR', 'WLD'), ('DOGE',
   'LINK'), ('BNB', 'LINK'), ('ETC', 'NEAR'), ('ADA', 'LTC')],
 (Timestamp('2024-05-04 00:00:00'),
  Timestamp('2024-05-06 23:59:00')): [('DOT', 'ETC'), ('ARB', 'DOT'), ('ADA',
   'LTC'), ('BTC', 'DOGE'), ('LTC', 'XLM')],
 (Timestamp('2024-05-05 00:00:00'),
  Timestamp('2024-05-07 23:59:00')): [('ADA', 'XLM'), ('ARB', 'DOT'), ('BCH',
   'ENA'), ('BTC', 'DOGE')],
 (Timestamp('2024-05-06 00:00:00'),
  Timestamp('2024-05-08 23:59:00')): [('BTC', 'DOGE'), ('APT', 'AVAX'), ('ARB',
   'DOT'), ('DOT', 'ETH'), ('BTC', 'SUI')],
 (Timestamp('2024-05-07 00:00:00'),
  Timestamp('2024-05

In [64]:
import pandas as pd
df = pd.read_csv("bin_futures_historical_pairs_with_spreads.csv",sep=";")
df
pd.date_range(start='2024-05-01 00:00:00', end='2025-05-01 00:00:01', freq='1T')
# create timestamp column from 2024-05-01 00:00:00 to 2025-05-01 00:00:00 with 1 min frequency
df['timestamp'] = pd.date_range(start='2024-05-01 00:00:00', end='2025-05-01 00:00:01', freq='1T')
df
# drop last observation to have exactly 525600 rows (1 year of minutely data)
df = df[:-1]

df
df.filter(like='spread').describe()

C:\Users\isaen\AppData\Local\Temp\ipykernel_17552\2154638657.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  pd.date_range(start='2024-05-01 00:00:00', end='2025-05-01 00:00:01', freq='1T')
C:\Users\isaen\AppData\Local\Temp\ipykernel_17552\2154638657.py:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df['timestamp'] = pd.date_range(start='2024-05-01 00:00:00', end='2025-05-01 00:00:01', freq='1T')


,AVAX_ETC_spread,XLM_XRP_spread,ETC_ETH_spread,ADA_XRP_spread,ADA_LTC_spread,NEAR_WLD_spread,ETC_NEAR_spread,ARB_ETC_spread,ARB_ETH_spread,DOGE_LINK_spread,...,ARB_WLD_spread,ATOM_ENA_spread,ADA_DOGE_spread,TRX_XLM_spread,DOGE_SUI_spread,AAVE_TRX_spread,LINK_UNI_spread,LINK_LTC_spread,TRX_XRP_spread,BTC_TRX_spread
count,136800.000000,146880.000000,141120.000000,115200.000000,106560.000000,171360.000000,149760.000000,195840.000000,129600.000000,144000.000000,...,152640.000000,61920.000000,103680.000000,27360.000000,64800.000000,20160.000000,90720.000000,82080.000000,34560.000000,31680.000000
mean,0.000183,0.000024,0.000183,0.000152,0.000026,0.000023,0.000138,0.000008,-0.000061,0.000107,...,-0.000094,-0.000087,0.000205,0.000077,0.000003,-0.001193,0.000075,0.000128,-0.000199,0.000071
std,0.009249,0.009311,0.007082,0.010253,0.009131,0.009825,0.007513,0.008119,0.007501,0.009650,...,0.007959,0.008726,0.011291,0.014975,0.010898,0.010931,0.010031,0.010074,0.014234,0.005003
min,-0.057036,-0.088194,-0.047637,-0.057757,-0.046375,-0.075698,-0.052571,-0.080632,-0.046434,-0.073028,...,-0.037776,-0.043272,-0.133322,-0.085998,-0.101073,-0.032968,-0.086718,-0.047966,-0.082842,-0.025135
25%,-0.004860,-0.004199,-0.003637,-0.005026,-0.005232,-0.005788,-0.004079,-0.004040,-0.004335,-0.004955,...,-0.005025,-0.005000,-0.004542,-0.003683,-0.005276,-0.008888,-0.005427,-0.005431,-0.005386,-0.002989
50%,0.000264,-0.000065,0.000050,-0.000326,0.000026,0.000092,-0.000050,-0.000006,-0.000170,0.000093,...,-0.000133,-0.000108,0.000328,-0.000037,0.000367,-0.000916,0.000295,0.000163,0.000326,-0.000041
75%,0.005333,0.003943,0.003754,0.005043,0.005150,0.005902,0.004270,0.004134,0.004186,0.005325,...,0.004709,0.005036,0.005004,0.003355,0.005130,0.006340,0.005956,0.006112,0.005193,0.003046
max,0.059225,0.101657,0.069126,0.098774,0.145970,0.071833,0.053366,0.089491,0.045405,0.101470,...,0.061839,0.047905,0.106183,0.196018,0.113285,0.035731,0.065791,0.059019,0.198323,0.021301


In [65]:
df.shape

(525600, 331)

In [67]:
spread_cols = [col for col in df.columns if ("alpha" in col) or ("beta" in col) or ("corr" in col) or ("adf" in col)]

In [68]:
spread_cols

['AVAX_ETC_beta',
 'AVAX_ETC_alpha',
 'AVAX_ETC_adf_p',
 'AVAX_ETC_corr',
 'XLM_XRP_beta',
 'XLM_XRP_alpha',
 'XLM_XRP_adf_p',
 'XLM_XRP_corr',
 'ETC_ETH_beta',
 'ETC_ETH_alpha',
 'ETC_ETH_adf_p',
 'ETC_ETH_corr',
 'ADA_XRP_beta',
 'ADA_XRP_alpha',
 'ADA_XRP_adf_p',
 'ADA_XRP_corr',
 'ADA_LTC_beta',
 'ADA_LTC_alpha',
 'ADA_LTC_adf_p',
 'ADA_LTC_corr',
 'NEAR_WLD_beta',
 'NEAR_WLD_alpha',
 'NEAR_WLD_adf_p',
 'NEAR_WLD_corr',
 'ETC_NEAR_beta',
 'ETC_NEAR_alpha',
 'ETC_NEAR_adf_p',
 'ETC_NEAR_corr',
 'ARB_ETC_beta',
 'ARB_ETC_alpha',
 'ARB_ETC_adf_p',
 'ARB_ETC_corr',
 'ARB_ETH_beta',
 'ARB_ETH_alpha',
 'ARB_ETH_adf_p',
 'ARB_ETH_corr',
 'DOGE_LINK_beta',
 'DOGE_LINK_alpha',
 'DOGE_LINK_adf_p',
 'DOGE_LINK_corr',
 'BNB_LINK_beta',
 'BNB_LINK_alpha',
 'BNB_LINK_adf_p',
 'BNB_LINK_corr',
 'DOT_ETC_beta',
 'DOT_ETC_alpha',
 'DOT_ETC_adf_p',
 'DOT_ETC_corr',
 'ARB_DOT_beta',
 'ARB_DOT_alpha',
 'ARB_DOT_adf_p',
 'ARB_DOT_corr',
 'BTC_DOGE_beta',
 'BTC_DOGE_alpha',
 'BTC_DOGE_adf_p',
 'BTC_DOGE

In [69]:
import pandas as pd
import numpy as np
import os

def normalize_spreads(full_df, top_pairs_per_window, folder_path="", trade_minutes=1440):
    """
    Normalize spreads for trading day using selection window beta/alpha 
    from per-pair CSVs and propagate beta/alpha/correlation to trade period.

    Parameters:
    - full_df: DataFrame with timestamp, price columns for all symbols
    - top_pairs_per_window: dict {(sel_start, sel_end): [('SYM1','SYM2'), ...]}
    - folder_path: folder containing per-pair CSVs named 
                   '{sym1}_{sym2}_bin_futures_window_cointegration.csv'
    - trade_minutes: int, duration of trading window (default 1 day)

    Returns:
    - DataFrame with new columns:
        '{pair}_spread_norm', '{pair}_alpha', '{pair}_beta', '{pair}_corr'
    """
    spread_cols = [col for col in full_df.columns if ("alpha" in col) or ("beta" in col) or ("corr" in col) or ("adf" in col)]
    df = full_df.drop(columns = spread_cols, axis = 1).copy()
    df = df.sort_values("timestamp").reset_index(drop=True)
    timestamps = df["timestamp"]

    for (sel_start, sel_end), top_pairs in top_pairs_per_window.items():
        # Selection window mask
        sel_mask = (timestamps >= sel_start) & (timestamps <= sel_end)
        if not any(sel_mask):
            print(f"No selection data for window {sel_start} to {sel_end}. Skipping.")
            continue

        # Trading window mask (next day after selection)
        trade_start = sel_end + pd.Timedelta(minutes=1)
        trade_end = trade_start + pd.Timedelta(minutes=trade_minutes - 1)
        trade_mask = (timestamps >= trade_start) & (timestamps <= trade_end)
        if not any(trade_mask):
            print(f"No trading data for window {sel_start} to {sel_end}. Skipping.")
            continue

        for sym1, sym2 in top_pairs:
            csv_file = os.path.join(folder_path, f"{sym1}_{sym2}_bin_futures_window_cointegration.csv")
            if not os.path.exists(csv_file):
                print(f"CSV not found for {sym1}-{sym2}. Skipping.")
                continue

            pair_df = pd.read_csv(csv_file, parse_dates=['start','end'])
            sel_row = pair_df[(pair_df['start'] == sel_start) & (pair_df['end'] == sel_end)]

            if sel_row.empty or not sel_row['cointegrated'].values[0]:
                continue

            alpha = sel_row['alpha'].values[0]
            beta = sel_row['beta'].values[0]
            corr = sel_row['correlation'].values[0]
            p_val = sel_row["adf_p"].values[0]

            if pd.isna(alpha) or pd.isna(beta):
                print(f"NaN alpha/beta for {sym1}-{sym2} in window {sel_start} to {sel_end}. Skipping.")
                continue

            # Compute selection window spread for mean/std
            y_sel = df.loc[sel_mask, f"{sym1}_close"]
            x_sel = df.loc[sel_mask, f"{sym2}_close"]
            sel_spread = y_sel - (alpha + beta * x_sel)
            mu = sel_spread.mean()
            sigma = sel_spread.std()
            if sigma == 0:
                sigma = 1e-9

            # Compute trading window spread and normalize
            y_trade = df.loc[trade_mask, f"{sym1}_close"]
            x_trade = df.loc[trade_mask, f"{sym2}_close"]
            if y_trade.isna().all() or x_trade.isna().all():
                print(f"All NaNs in prices for {sym1}-{sym2} in window {trade_start} to {trade_end}. Skipping.")
                continue

            trade_spread = y_trade - (alpha + beta * x_trade)
            if trade_spread.isna().any():
                print(f"NaNs in trade spread for {sym1}-{sym2} in window {trade_start} to {trade_end}")

            # Column names
            spread_norm_col = f"{sym1}_{sym2}_spreadNorm"
            alpha_col = f"{sym1}_{sym2}_alpha"
            beta_col = f"{sym1}_{sym2}_beta"
            corr_col = f"{sym1}_{sym2}_corr"
            pval_col = f"{sym1}_{sym2}_pval"

            # Store normalized spread
            df.loc[trade_mask, spread_norm_col] = (trade_spread - mu) / sigma

            # 🔥 Store alpha, beta, correlation for the trade period
            df.loc[trade_mask, alpha_col] = alpha
            df.loc[trade_mask, beta_col] = beta
            df.loc[trade_mask, corr_col] = corr
            df.loc[trade_mask, pval_col] = p_val

    return df


In [70]:
df

,AAVE_close,ADA_close,APT_close,ARB_close,ATOM_close,AVAX_close,BCH_close,BNB_close,BTC_close,DOGE_close,...,TRX_XRP_beta,TRX_XRP_alpha,TRX_XRP_adf_p,TRX_XRP_corr,BTC_TRX_spread,BTC_TRX_beta,BTC_TRX_alpha,BTC_TRX_adf_p,BTC_TRX_corr,timestamp
0,4.423169,-0.818257,2.166078,0.021370,2.136176,3.486794,6.072814,6.358864,11.013869,-2.014703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-01 00:00:00
1,4.422809,-0.819164,2.164012,0.020195,2.138418,3.485937,6.071938,6.357825,11.014249,-2.014703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-01 00:01:00
2,4.421368,-0.819845,2.163553,0.018527,2.136294,3.485630,6.070992,6.357634,11.012786,-2.016504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-01 00:02:00
3,4.422929,-0.818937,2.167223,0.020097,2.136176,3.486304,6.072422,6.357409,11.013483,-2.015528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-01 00:03:00
4,4.422569,-0.818937,2.166651,0.019508,2.135349,3.485937,6.071476,6.357201,11.012911,-2.015303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525595,5.100171,-0.383899,1.676592,-1.122700,1.456986,3.038313,5.901403,6.394894,11.452122,-1.759563,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-30 23:55:00
525596,5.100232,-0.383899,1.675488,-1.122393,1.457452,3.039271,5.902415,6.394978,11.452460,-1.759157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-30 23:56:00
525597,5.101085,-0.383753,1.676143,-1.121779,1.458150,3.039845,5.902469,6.395746,11.452573,-1.758750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-30 23:57:00
525598,5.101329,-0.383753,1.676405,-1.121165,1.457917,3.039893,5.902415,6.395762,11.452698,-1.758808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-30 23:58:00


In [71]:
full_df_norm = normalize_spreads(df, top_pairs_per_window)


C:\Users\isaen\AppData\Local\Temp\ipykernel_17552\2828878547.py:95: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[trade_mask, corr_col] = corr
C:\Users\isaen\AppData\Local\Temp\ipykernel_17552\2828878547.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[trade_mask, pval_col] = p_val
C:\Users\isaen\AppData\Local\Temp\ipykernel_17552\2828878547.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

No trading data for window 2025-04-28 00:00:00 to 2025-04-30 23:59:00. Skipping.


In [72]:
full_df_norm.shape
# 25 + 61*6 + 1

(525600, 392)

In [73]:
full_df_norm["AAVE_SUI_alpha"].notna().sum(),  full_df_norm["AAVE_SUI_spreadNorm"].notna().sum()

(27360, 27360)

In [74]:
import pandas as pd

def check_pair_data_consistency(df):
    """
    Checks that for each pair (e.g. AAVE_SUI) all related columns
    ('alpha', 'beta', 'spread', 'spread_norm', etc.) have the same number of non-NaN values.
    Prints inconsistencies if found.
    """
    # extract unique pair prefixes before last underscore
    pair_names = sorted({'_'.join(col.split('_')[:2]) for col in df.columns if '_' in col})

    inconsistencies = []

    for pair in pair_names:
        # all columns for this pair
        pair_cols = [col for col in df.columns if col.startswith(pair + "_") and not col.endswith("spread")]
        if not pair_cols:
            continue

        # count non-NaN per column
        counts = {col: df[col].notna().sum() for col in pair_cols}

        # check if all counts are equal
        unique_counts = set(counts.values())
        if len(unique_counts) > 1:
            inconsistencies.append((pair, counts))

    if inconsistencies:
        print("❌ Inconsistencies found:")
        for pair, counts in inconsistencies:
            print(f"\nPair: {pair}")
            for col, count in counts.items():
                print(f"  {col}: {count}")
    else:
        print("✅ All pair columns have consistent non-NaN counts.")

# Example usage
check_pair_data_consistency(full_df_norm)


✅ All pair columns have consistent non-NaN counts.


In [75]:
(full_df_norm.loc[:4319].filter(like='spread').isna().sum() != 4320).sum()

11

In [76]:
525600 - 4320

521280

In [77]:
full_df_norm.filter(like='spreadNorm').notna().sum(axis=1)

0         0
1         0
2         0
3         0
4         0
         ..
525595    5
525596    5
525597    5
525598    5
525599    5
Length: 525600, dtype: int64

In [78]:
# Get all normalized spread columns
norm_cols = [c for c in full_df_norm.columns if "_spreadNorm" in c]

# Boolean Series: True if at least one normalized spread is not NaN
row_has_no_data = full_df_norm[~full_df_norm[norm_cols].notna().any(axis=1)]

# Check
print(f"Rows with no _spreadNorm value: {len(row_has_no_data)} / {len(full_df_norm)}")

# Optional: see rows with no valid normalized spreads
# rows_no_norm = full_df_norm.loc[~row_has_data]
# rows_no_norm
row_has_no_data
# 4320 for first 3 days + 1440 for 21.08 (because for 18.08-20.08 there is no cointegrated pairs)

Rows with no _spreadNorm value: 5760 / 525600


,AAVE_close,ADA_close,APT_close,ARB_close,ATOM_close,AVAX_close,BCH_close,BNB_close,BTC_close,DOGE_close,...,TRX_XRP_spreadNorm,TRX_XRP_alpha,TRX_XRP_beta,TRX_XRP_corr,TRX_XRP_pval,BTC_TRX_spreadNorm,BTC_TRX_alpha,BTC_TRX_beta,BTC_TRX_corr,BTC_TRX_pval
0,4.423169,-0.818257,2.166078,0.021370,2.136176,3.486794,6.072814,6.358864,11.013869,-2.014703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.422809,-0.819164,2.164012,0.020195,2.138418,3.485937,6.071938,6.357825,11.014249,-2.014703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.421368,-0.819845,2.163553,0.018527,2.136294,3.485630,6.070992,6.357634,11.012786,-2.016504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.422929,-0.818937,2.167223,0.020097,2.136176,3.486304,6.072422,6.357409,11.013483,-2.015528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.422569,-0.818937,2.166651,0.019508,2.135349,3.485937,6.071476,6.357201,11.012911,-2.015303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162715,4.943355,-0.996959,1.848297,-0.592759,1.557091,3.157596,5.854929,6.345303,11.021208,-2.244977,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162716,4.942071,-0.997772,1.848140,-0.593121,1.556881,3.157341,5.855015,6.345145,11.020787,-2.243091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162717,4.942428,-0.997230,1.848297,-0.593483,1.556670,3.157596,5.854957,6.345075,11.020830,-2.243185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162718,4.941714,-0.998043,1.848455,-0.593302,1.556459,3.157553,5.854900,6.345057,11.020552,-2.243185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
(row_has_no_data["timestamp"][:4320] == pd.date_range(start='2024-05-01 00:00:00', periods=4320, freq='1T')).sum()

C:\Users\isaen\AppData\Local\Temp\ipykernel_17552\1225828755.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  (row_has_no_data["timestamp"][:4320] == pd.date_range(start='2024-05-01 00:00:00', periods=4320, freq='1T')).sum()


4320

In [80]:
row_has_no_data["timestamp"][4320:]
# 21.08.2024 - something wrong there

161280   2024-08-21 00:00:00
161281   2024-08-21 00:01:00
161282   2024-08-21 00:02:00
161283   2024-08-21 00:03:00
161284   2024-08-21 00:04:00
                 ...        
162715   2024-08-21 23:55:00
162716   2024-08-21 23:56:00
162717   2024-08-21 23:57:00
162718   2024-08-21 23:58:00
162719   2024-08-21 23:59:00
Name: timestamp, Length: 1440, dtype: datetime64[ns]

In [81]:
full_df_norm[(full_df_norm["timestamp"] >= pd.to_datetime('2024-08-21')) & (full_df_norm["timestamp"] < pd.to_datetime('2024-08-22'))].filter(like='spread').notna().sum(axis=1)

161280    9
161281    9
161282    9
161283    9
161284    9
         ..
162715    9
162716    9
162717    9
162718    9
162719    9
Length: 1440, dtype: int64

In [82]:
full_df_norm

,AAVE_close,ADA_close,APT_close,ARB_close,ATOM_close,AVAX_close,BCH_close,BNB_close,BTC_close,DOGE_close,...,TRX_XRP_spreadNorm,TRX_XRP_alpha,TRX_XRP_beta,TRX_XRP_corr,TRX_XRP_pval,BTC_TRX_spreadNorm,BTC_TRX_alpha,BTC_TRX_beta,BTC_TRX_corr,BTC_TRX_pval
0,4.423169,-0.818257,2.166078,0.021370,2.136176,3.486794,6.072814,6.358864,11.013869,-2.014703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.422809,-0.819164,2.164012,0.020195,2.138418,3.485937,6.071938,6.357825,11.014249,-2.014703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.421368,-0.819845,2.163553,0.018527,2.136294,3.485630,6.070992,6.357634,11.012786,-2.016504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.422929,-0.818937,2.167223,0.020097,2.136176,3.486304,6.072422,6.357409,11.013483,-2.015528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.422569,-0.818937,2.166651,0.019508,2.135349,3.485937,6.071476,6.357201,11.012911,-2.015303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525595,5.100171,-0.383899,1.676592,-1.122700,1.456986,3.038313,5.901403,6.394894,11.452122,-1.759563,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525596,5.100232,-0.383899,1.675488,-1.122393,1.457452,3.039271,5.902415,6.394978,11.452460,-1.759157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525597,5.101085,-0.383753,1.676143,-1.121779,1.458150,3.039845,5.902469,6.395746,11.452573,-1.758750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525598,5.101329,-0.383753,1.676405,-1.121165,1.457917,3.039893,5.902415,6.395762,11.452698,-1.758808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
full_df_norm.to_csv("bin_futures_historical_pairs_with_spreads_normalized.csv", sep=";", index=False)